# EasyCV图像检测-YOLOX
本文将以[YOLOX](https://arxiv.org/abs/2107.08430)模型为例，介绍如何基于easyCV进行目标检测模型训练和预测

## 运行环境要求

PAI-Pytorch镜像 or 原生Pytorch1.5+以上环境 GPU机器， 内存32G以上

## 安装依赖包

注: 在PAI-DSW docker中无需安装相关依赖，可跳过此部分 在本地notebook环境中执行


1、 首先，安装pytorch和对应版本的torchvision，支持Pytorch1.5.1以上版本

In [ ]:
# install pytorch and torch vision
! conda install --yes pytorch==1.10.0 torchvision==0.11.0 -c pytorch

2、 获取torch和cuda版本，安装对应版本的mmcv和nvidia-dali

In [ ]:
import torch
import os
os.environ['CUDA']='cu' + torch.version.cuda.replace('.', '')
os.environ['Torch']='torch'+torch.version.__version__.replace('+PAI', '')
!echo "cuda version: $CUDA"
!echo "pytorch version: $Torch"

In [ ]:
# install some python deps
! pip install mmcv-full==1.4.4 -f https://download.openmmlab.com/mmcv/dist/${CUDA}/${Torch}/index.html
! pip install http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/third_party/nvidia_dali_cuda100-0.25.0-1535750-py3-none-manylinux2014_x86_64.whl

3、  安装EasyCV算法包

In [ ]:
pip install pai-easycv

4、 简单验证

In [ ]:
from easycv.apis import *

## 图像检测模型训练&预测

### 数据准备

你可以下载[COCO2017](https://cocodataset.org/#download)数据，也可以使用我们提供了示例COCO数据

In [ ]:
! wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/small_coco_demo/small_coco_demo.tar.gz && tar -zxf small_coco_demo.tar.gz

重命名数据文件，使其和COCO数据格式完全一致

In [ ]:
!mkdir -p data/  && mv small_coco_demo data/coco

data/coco格式如下

```shell
data/coco/
├── annotations
│   ├── instances_train2017.json
│   └── instances_val2017.json
├── train2017
│   ├── 000000005802.jpg
│   ├── 000000060623.jpg
│   ├── 000000086408.jpg
│   ├── 000000118113.jpg
│   ├── 000000184613.jpg
│   ├── 000000193271.jpg
│   ├── 000000222564.jpg
│       ...
│   └── 000000574769.jpg
└── val2017
    ├── 000000006818.jpg
    ├── 000000017627.jpg
    ├── 000000037777.jpg
    ├── 000000087038.jpg
    ├── 000000174482.jpg
    ├── 000000181666.jpg
    ├── 000000184791.jpg
    ├── 000000252219.jpg
         ...
    └── 000000522713.jpg
```

### 模型训练

下载示例配置文件， 进行YOLOX-S模型训练

In [ ]:
! rm -rf yolox_s_8xb16_300e_coco.py
! wget https://raw.githubusercontent.com/alibaba/EasyCV/master/configs/detection/yolox/yolox_s_8xb16_300e_coco.py

为了适配小数据，我们对配置文件yolox_s_8xb16_300e_coco.py做如下字段的修改，减少训练epoch数目，加大打印日志的频率

```python

total_epochs = 3

#optimizer.lr -> 0.0002
optimizer = dict(
    type='SGD', lr=0.0002, momentum=0.9, weight_decay=5e-4, nesterov=True)

# log_config.interval 1
log_config = dict(interval=1)

```

注意： 如果是使用COCO完整数据训练，为了保证效果，建议使用单机8卡进行训练；  如果要使用单卡训练，建议降低学习率`optimizer.lr`

为了保证模型效果，我们在[预训练模型](http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/EasyCV/modelzoo/detection/yolox_s_bs16_lr002/epoch_300.pth)基础上finetune， 执行如下命令启动训练

In [ ]:
!python -m easycv.tools.train yolox_s_8xb16_300e_coco.py --work_dir work_dir/detection/yolox/yolox_s_8xb16_300e_coco --load_from http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/EasyCV/modelzoo/detection/yolox_s_bs16_lr002/epoch_300.pth

### 导出模型
导出YOLOX 模型用于预测， 执行如下命令查看训练产生的模型文件

In [ ]:
! ls work_dir/detection/yolox/yolox_s_8xb16_300e_coco/*.pth

在导出模型前，需要对配置文件进行修改，指定nms的得分阈值

model.test_conf 0.01 -> 0.5

```python
model = dict(
    type='YOLOX',
    num_classes=80,
    model_type='s',  # s m l x tiny nano
    test_conf=0.5,
    nms_thre=0.65)
```

执行如下命令进行模型导出

In [ ]:
! cp yolox_s_8xb16_300e_coco.py yolox_s_8xb16_300e_coco_export.py && sed -i 's#test_conf=0.01#test_conf=0.5#g' yolox_s_8xb16_300e_coco_export.py
!python -m easycv.tools.export yolox_s_8xb16_300e_coco_export.py work_dir/detection/yolox/yolox_s_8xb16_300e_coco/epoch_30.pth  work_dir/detection/yolox/yolox_s_8xb16_300e_coco/yolox_export.pth

### 模型预测
下载测试图片

In [ ]:
!wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/small_coco_demo/val2017/000000017627.jpg

In [ ]:
import cv2
from easycv.predictors import TorchYoloXPredictor

output_ckpt = 'work_dir/detection/yolox/yolox_s_8xb16_300e_coco/yolox_export.pth'
detector = TorchYoloXPredictor(output_ckpt)

img = cv2.imread('000000017627.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
output = detector.predict([img])
print(output)

In [ ]:
# view detection results

%matplotlib inline
from matplotlib import pyplot as plt
image = img.copy()
for box, cls_name in zip(output[0]['detection_boxes'], output[0]['detection_class_names']):
    # box is [x1,y1,x2,y2]
    box = [int(b) for b in box]
    image = cv2.rectangle(image, tuple(box[:2]), tuple(box[2:4]), (0,255,0), 2)
    cv2.putText(image, cls_name, (box[0], box[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)
plt.imshow(image)
plt.show()